In [16]:
import xml.etree.ElementTree as ET
import mysql.connector
from decimal import Decimal

# Function to calculate the cost of a specific unit operation
def calculate_unit_operation_cost(operation_element, db_cursor):
    operation_name = operation_element.tag
    query = f"SELECT UOCost, WorkCost FROM UnitOperations WHERE UOName = %s"

    db_cursor.execute(query, (operation_name,))
    result = db_cursor.fetchone()

    if result:
        uo_cost, work_cost = result
        return float(uo_cost + work_cost)
    else:
        return 0.0  # Default cost if operation not found in the database

# Function to calculate the total cost of a sequence
def calculate_sequence_cost(sequence_element, db_cursor):
    total_cost = 0.0

    for unit_operation_element in sequence_element:
        if unit_operation_element.tag == "UnitOperation":
            cost = calculate_unit_operation_cost(unit_operation_element[0], db_cursor)
            total_cost += cost
            print(f"Unit Operation: {unit_operation_element[0].tag}, Cost: {cost:.2f}")

    return total_cost

def main():
    tree = ET.parse('/content/3UnitOperations.xml')  # Replace with your XML file path
    root = tree.getroot()

    # Connect to your MySQL database
    db_connection = mysql.connector.connect(
        host='host',
        user='user',
        password='password',
        database='db'
    )
    db_cursor = db_connection.cursor()

    total_sequence_cost = calculate_sequence_cost(root, db_cursor)

    # Close the database connection
    db_cursor.close()
    db_connection.close()

    print(f"Total cost of the sequence: {total_sequence_cost:.2f}")

if __name__ == "__main__":
    main()


Unit Operation: Reaction, Cost: 550.00
Unit Operation: AddOnce, Cost: 0.00
Unit Operation: Dry, Cost: 125.00
Total cost of the sequence: 675.00
